# Word Embeddings : le modèle Word2Vec

## Imports

In [3]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [4]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [5]:
infile = "/System/Volumes/Data/Users/azossieteresafabiola/Tp3/tac/tps/data/txt/sents.txt"
sentences = MySentences(infile)
print(type(sentences))

import os
print(infile)
print(os.path.exists(infile))

<class '__main__.MySentences'>
/System/Volumes/Data/Users/azossieteresafabiola/Tp3/tac/tps/data/txt/sents.txt
True


In [6]:
infile = "/System/Volumes/Data/Users/azossieteresafabiola/Tp3/tac/tps/data/txt/sents.txt"
sentences = MySentences(infile)

In [7]:
import os
print(os.getcwd())

/Users/azossieteresafabiola/Tp3/tac/tps/tp3


In [8]:
for root, dirs, files in os.walk(os.getcwd(), topdown=True):
    if "sents.txt" in files:
        print("FOUND:", os.path.join(root, "sents.txt"))

In [9]:
import os

for root, dirs, files in os.walk("/", topdown=True):
    if "sents.txt" in files:
        print("FOUND:", os.path.join(root, "sents.txt"))
        break

FOUND: /System/Volumes/Data/Users/azossieteresafabiola/Tp3/tac/tps/data/txt/sents.txt


In [11]:
infile = "/System/Volumes/Data/Users/azossieteresafabiola/Tp3/tac/tps/data/txt/sents.txt"
sentences = MySentences(infile)

import os
print(infile)
print(os.path.exists(infile))

/System/Volumes/Data/Users/azossieteresafabiola/Tp3/tac/tps/data/txt/sents.txt
True


### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [12]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [13]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [14]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [15]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [16]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [17]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [18]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [19]:

trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [20]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [21]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [22]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 37min 16s, sys: 31min 31s, total: 1h 8min 47s
Wall time: 2h 41min 8s


In [34]:
outfile = "../data/newspapers.model"
model.save(outfile)

#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [23]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [36]:
import os

print("CWD =", os.getcwd())  # où tu es exactement
for root, dirs, files in os.walk(os.getcwd()):
    if "newspapers.model" in files:
        print("FOUND MODEL:", os.path.join(root, "newspapers.model"))

CWD = /Users/azossieteresafabiola/Tp3/tac/tps/tp3


### Imprimer le vecteur d'un terme

In [25]:
model.wv["ministre"]

array([ -1.5731053 ,  -2.4102094 ,   1.1297305 ,   5.180584  ,
        -5.0943117 ,  -2.3847933 ,  -7.052009  ,  -4.413669  ,
        -4.570704  ,  -2.303178  ,   1.2155125 ,  -2.4840639 ,
        -3.191879  ,  -5.892077  ,  -1.0825129 ,   3.6056783 ,
        -5.6272354 ,  -3.8113697 ,  -0.17154323,  -8.60715   ,
        -5.8219314 ,  -1.3595284 ,   4.434311  ,  -2.591416  ,
        -2.3032727 ,  -3.8585687 ,  -2.0520978 ,  -6.152695  ,
         4.6079407 , -11.851414  ,  -3.2925901 ,  -1.3984473 ],
      dtype=float32)

### Calculer la similarité entre deux termes

In [26]:
model.wv.similarity("ministre", "roi")

0.7236861

### Chercher les mots les plus proches d'un terme donné

In [27]:
model.wv.most_similar("ministre", topn=10)

[('premier_ministre', 0.9182556867599487),
 ('ministre_des_finances', 0.8865057229995728),
 ('ministere', 0.8847351670265198),
 ('rapporteur', 0.874431312084198),
 ('chancelier', 0.8742679953575134),
 ('president', 0.8686789870262146),
 ('mi_nistre', 0.8564624190330505),
 ('ministro', 0.8497983813285828),
 ('gouverneur_general', 0.8491296768188477),
 ('ancien_ministre', 0.8409953117370605)]

### Faire des recherches complexes à travers l'espace vectoriel

In [31]:
print(model.wv.most_similar(positive=['paris','londres','allemagne'], negative=['belgique'], topn=10))

[('berlin', 0.936758816242218), ('rome', 0.9005275964736938), ('moscou', 0.8978778719902039), ('geneve', 0.8574203252792358), ('vienne', 0.8552701473236084), ('budapest', 0.8532125949859619), ('alger', 0.8517098426818848), ('prague', 0.8445860743522644), ('washington', 0.8389530777931213), ('stockholm', 0.8375347256660461)]


In [33]:
print(model.wv.most_similar(positive=['paris', 'londres','allemagne'], negative=['france', 'angleterre'], topn=10))

[('berne', 0.8363231420516968), ('stockholm', 0.8158223032951355), ('berlin', 0.8101850748062134), ('fevrier', 0.7998620271682739), ('decembre', 0.7931808233261108), ('budapest', 0.7913472056388855), ('janvier', 0.7873720526695251), ('octobre', 0.7792397737503052), ('novembre', 0.7785266041755676), ('copenhague', 0.7749161720275879)]
